Беглым поиском нашелся только [минимальный датасет](https://huggingface.co/huggingartists/vladimir-vysotsky) на 47 произведений, включая явно посторонние произведения.
- нужен парсер

In [8]:
import json

with open("./datasets/vysotskiy.txt") as f:
    texts = json.load(f)

len(texts["train"])

47

# Парсер

- http://vysotskiy-lit.ru/vysotskiy/stihi/stihi-all.htm
- собрано 764 произведений (поэзия)

In [9]:
import os
import time
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [10]:
LYRICS_DUMP = "./datasets/vysotskiy_all.json"

def parse_lyrics(link):
    xpath_song_link = "//*[@id='main_text_box']//li/a"
    xpath_lyrics = "(//*[@id='main_text_box']//p)[last()-1]"
    xpath_year = "(//*[@id='main_text_box']//p)[last()]"
    
    # смотрим что уже скачалось
    if os.path.isfile(LYRICS_DUMP):
        with open(LYRICS_DUMP) as f:
            songs = json.load(f)
    else:
        songs = {}

    with webdriver.Chrome() as driver:
        driver.get(link)
        names_urls = driver.find_elements(By.XPATH, xpath_song_link)
        
        # фильтр уже скачанных
        names_urls = [(s.text, s.get_attribute('href')) for s in names_urls]
        names_urls = [n_u for n_u in names_urls if not songs.get(n_u[1], False) or not songs[n_u[1]].get("done", False)]
        print(f"Resume from: {names_urls[0]}")

        for song in tqdm(names_urls):
            name, url = song
            songs[url] = {"name": name, "done": False}

            driver.get(url)
            try:
                songs[url]["lyrics"] = driver.find_element(By.XPATH, xpath_lyrics).text
                songs[url]["year"] = driver.find_element(By.XPATH, xpath_year).text
            except NoSuchElementException:
                print(f"SKIP\tError parsing: {name} - {url}")
                continue

            songs[url]["done"] = True
            time.sleep(3)
                 
            # обновить дамп, если спарсилось ОК
            if songs[url].get("done", False):
                with open(LYRICS_DUMP, "w") as f:
                    json.dump(songs, f)


link = "http://vysotskiy-lit.ru/vysotskiy/stihi/stihi-all.htm"

# parse_lyrics(link)

In [11]:
with open(LYRICS_DUMP) as f:
    texts = json.load(f)

done = sum([v["done"] for k, v in texts.items()])
total = len(texts.keys())

print(f"{total=}\n{done=}\nerrors={total-done}")

total=764
done=764
errors=0
